## Import Library and Load data

In [1]:
#Modifikasi
import warnings
import zipfile
import numpy as np
import pandas as pd
from pathlib import Path
pd.set_option('display.max_columns', 100)

#Perhitungan
from sklearn.preprocessing import PolynomialFeatures

import matplotlib.pyplot as plt

#Imputasi
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression


# Modeling
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV, GroupKFold,KFold, TimeSeriesSplit   
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc,roc_auc_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, RepeatVector, TimeDistributed
from feature_engine.creation import CyclicalFeatures


#Feature Selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Feature Importance
from sklearn.ensemble import ExtraTreesClassifier

In [2]:
train = pd.read_csv('raw_dataset/Train.csv')
test = pd.read_csv('raw_dataset/Test.csv')

train.drop(['ID','ID_Zindi'], axis=1, inplace=True)
test.drop(['ID','ID_Zindi'], axis=1, inplace=True)

## Data Cleaning

In [3]:
train['Date'] = pd.to_datetime(train['Date'], dayfirst=True, errors='coerce')

Date = train.copy()
Date['Date'] = pd.to_datetime(Date['Date'])


# 3. Menetapkan kolom Date sebagai index
train.set_index('Date', inplace=True)

C:\Users\booma\AppData\Local\Temp\ipykernel_27244\2488958140.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train['Date'] = pd.to_datetime(train['Date'], dayfirst=True, errors='coerce')


In [4]:
test['Date'] = pd.to_datetime(test['Date'], dayfirst=True, errors='coerce')

Date_test = test.copy()
Date_test['Date'] = pd.to_datetime(Date['Date'])


# 3. Menetapkan kolom Date sebagai index
test.set_index('Date', inplace=True)

C:\Users\booma\AppData\Local\Temp\ipykernel_27244\1334361193.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  test['Date'] = pd.to_datetime(test['Date'], dayfirst=True, errors='coerce')


## Imputasi

In [5]:
# df = train.copy()
# df.drop(columns=['ID_Zindi','ID'],inplace=True)
# test.drop(columns=['ID_Zindi','ID'],inplace=True)

# def impute_missing_values(df, cols_to_impute, drop_cols=['LAT', 'LON'], n_estimators=100, random_state=42):
#     for col in cols_to_impute:
#         if df[col].isna().sum() > 0:  # Cek apakah ada nilai NaN pada kolom
#             non_missing_data = df[df[col].notna()]  # Data tanpa nilai NaN untuk training
#             X_train = non_missing_data.drop(columns=[col] + drop_cols)  # Fitur training tanpa kolom target
#             y_train = non_missing_data[col]  # Target untuk training
            
#             # Inisiasi Random Forest Regressor dan training
#             rf_imputer = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
#             rf_imputer.fit(X_train, y_train)
            
#             # Melakukan prediksi untuk mengisi nilai NaN
#             X_pred = df[df[col].isna()].drop(columns=[col] + drop_cols)
#             df.loc[df[col].isna(), col] = rf_imputer.predict(X_pred)
    
#     return df


# cols_to_impute_rf = ['AAI', 'CloudFraction','LST', 'NO2_trop', 'NO2_strat', 'NO2_total', 'TropopausePressure']
# df = impute_missing_values(df, cols_to_impute_rf)
# test = impute_missing_values(test, cols_to_impute_rf)

# # Imputasi untuk kolom dengan missing data sedikit (Mean Imputation)
# cols_to_impute_mean = ['GT_NO2']
# mean_imputer = SimpleImputer(strategy='mean')
# df[cols_to_impute_mean] = mean_imputer.fit_transform(df[cols_to_impute_mean])

# # Time series imputation using Iterative Imputer (Multiple Imputation)
# time_series_cols = ['Precipitation']
# time_series_imputer = IterativeImputer(random_state=42)
# df[time_series_cols] = time_series_imputer.fit_transform(df[time_series_cols])
# test[time_series_cols] = time_series_imputer.fit_transform(test[time_series_cols])

# Feature Engineering

In [41]:
# df.to_csv('train_imputed_with_rf_regressor.csv', index=False)
# test.to_csv('test_imputed_with_rf_regressor.csv', index=False)

data = pd.read_csv('final_dataset/train_imputed_with_rf_regressor.csv')
data.drop(['LAT','LON'],axis=1,inplace=True)


In [42]:
dtest = pd.read_csv('final_dataset/test_imputed_with_rf_regressor.csv')

dtest.drop(['LAT','LON'],axis=1,inplace=True)

In [43]:
data.shape, train.shape

((86584, 9), (86584, 11))

In [44]:
kmeans = KMeans(n_clusters=3)
data['Kmeans'] = kmeans.fit_predict(data[['NO2_strat', 'NO2_total', 'NO2_trop']])

scaler = StandardScaler()
data['TropopausePressure'] = scaler.fit_transform(data[['TropopausePressure']])

# Misalnya, data berisi kolom 'Precipitation', 'LST', 'AAI'
poly = PolynomialFeatures(degree=2)

# Menerapkan transformasi polinomial pada kolom-kolom yang ditentukan
poly_features = poly.fit_transform(data[['Precipitation', 'LST', 'AAI']])

# Membuat DataFrame baru dengan nama kolom yang sesuai
poly_feature_columns = poly.get_feature_names_out(['Precipitation', 'LST', 'AAI'])

# Menggabungkan hasilnya dengan data asli (jika perlu)
poly_data = pd.DataFrame(poly_features, columns=poly_feature_columns)

poly_data.drop(['1','Precipitation','LST','AAI',],axis=1,inplace=True)

result = pd.concat([data, poly_data], axis=1)

result['GT_NO2'] = pd.to_numeric(result['GT_NO2'], errors='coerce')  # Pastikan target numerik

result["month"] = train.index.month
result["year"] = train.index.year
result['day'] = train.index.day

# Sinusoidal Encoding Bulan ``
result['month_Sin'] = np.sin(2 * np.pi * result['month'] / 12)
result['month_Cos'] = np.cos(2 * np.pi * result['month'] / 12)

# Fitur Elapsed months (pastikan tahun dimulai dari min tahun di result)
min_year = result['year'].min()
result['Elapsed_months'] = (result['year'] - min_year) * 12 + result['month']

# Quarter & Semester
result['Quarter'] = ((result['month'] - 1) // 3 + 1).astype(int)

result['Semester'] = ((result['month'] - 1) // 6 + 1).astype(int)

# year-month Identifier (Format YYYYMM)
result['yearmonth'] = (result['year'] * 100 + result['month']).astype(int)

In [45]:
target_col = 'GT_NO2'  # Nama kolom target
cols = [col for col in result.columns if col != target_col] + [target_col]
result = result[cols]

In [47]:
result.head()

,Precipitation,LST,AAI,CloudFraction,NO2_strat,NO2_total,NO2_trop,TropopausePressure,Kmeans,Precipitation^2,Precipitation LST,Precipitation AAI,LST^2,LST AAI,AAI^2,month,year,day,month_Sin,month_Cos,Elapsed_months,Quarter,Semester,yearmonth,GT_NO2
0,0.000000,277.4618,0.230527,0.559117,0.000024,0.000117,0.000163,-0.858776,1,0.000000,0.000000,0.000000,76985.050459,63.962302,0.053142,1,2019,1,0.5,0.866025,1,1,1,201901,31.0
1,3.047342,277.9274,-0.074006,0.869309,0.000024,0.000127,0.000123,-0.858437,1,9.286295,846.939922,-0.225522,77243.639671,-20.568365,0.005477,1,2019,1,0.5,0.866025,1,1,1,201901,42.0
2,0.000000,277.1622,0.024470,0.674160,0.000024,0.000086,0.000089,-0.859968,1,0.000000,0.000000,0.000000,76818.885109,6.782106,0.000599,1,2019,1,0.5,0.866025,1,1,1,201901,31.0
3,1.200467,277.4052,-0.010442,0.920054,0.000024,0.000124,0.000123,-0.858770,1,1.441121,333.015752,-0.012535,76953.644987,-2.896627,0.000109,1,2019,1,0.5,0.866025,1,1,1,201901,30.0
4,1.274564,278.9034,-0.176178,0.747464,0.000024,0.000116,0.000164,-0.859480,1,1.624513,355.480174,-0.224550,77787.106532,-49.136681,0.031039,1,2019,1,0.5,0.866025,1,1,1,201901,58.0


# Model Creation

In [18]:
import optuna
# model
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

#### LightGBM

In [50]:
# Fungsi evaluasi MSE
def mse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

# Definisikan objective function untuk Optuna
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50)
    }
    
    # Out-of-fold prediction
    oof_predictions = np.zeros(len(result))
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    for fold, (data_idx, val_idx) in enumerate(kf.split(result.drop('GT_NO2', axis=1))):
        X_data, X_val = result.drop('GT_NO2', axis=1).iloc[data_idx], result.drop('GT_NO2', axis=1).iloc[val_idx]
        y_data, y_val = result['GT_NO2'].iloc[data_idx], result['GT_NO2'].iloc[val_idx]
        
        model = LGBMRegressor(**params, random_state=42)
        model.fit(X_data, y_data)
        
        oof_predictions[val_idx] = model.predict(X_val)
    
    return mse(result['GT_NO2'], oof_predictions)

# Lakukan optimasi hyperparameter
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Gunakan parameter terbaik untuk model final
best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

lgbm = LGBMRegressor(**best_params, random_state=42)
y = result['GT_NO2']

# Out-of-fold predictions dengan parameter terbaik
oof_predictions = np.zeros(len(result))
kf = KFold(n_splits=10, shuffle=True, random_state=42)
fold_scores = []

for fold, (data_idx, val_idx) in enumerate(kf.split(result.drop('GT_NO2', axis=1))):
    X_data, X_val = result.drop('GT_NO2', axis=1).iloc[data_idx], result.drop('GT_NO2', axis=1).iloc[val_idx]
    y_data, y_val = result['GT_NO2'].iloc[data_idx], result['GT_NO2'].iloc[val_idx]
    
    lgbm.fit(X_data, y_data)
    val_pred = lgbm.predict(X_val)
    oof_predictions[val_idx] = val_pred
    
    fold_mse = mse(y_val, val_pred)
    fold_scores.append(fold_mse)
    print(f"Fold {fold+1} MSE: {fold_mse:.4f}")

# Menampilkan Rata-rata MSE
print(f"\nRata-rata MSE: {np.mean(fold_scores):.4f}")


[I 2025-02-14 11:17:51,170] A new study created in memory with name: no-name-de18edba-7850-48fc-8a7d-59112c01a9c8


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005064 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3725
[LightGBM] [Info] Number of data points in the train set: 77925, number of used features: 24
[LightGBM] [Info] Start training from score 24.539766
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

[I 2025-02-14 11:18:01,315] Trial 0 finished with value: 105.60658443865938 and parameters: {'n_estimators': 202, 'max_depth': 3, 'learning_rate': 0.18746675872187465, 'subsample': 0.5687499482347409, 'colsample_bytree': 0.746896531506422, 'min_child_samples': 25}. Best is trial 0 with value: 105.60658443865938.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008882 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3725
[LightGBM] [Info] Number of data points in the train set: 77925, number of used features: 24
[LightGBM] [Info] Start training from score 24.539766
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warni

[I 2025-02-14 11:18:09,133] Trial 1 finished with value: 103.63078717089086 and parameters: {'n_estimators': 66, 'max_depth': 5, 'learning_rate': 0.12039343138190267, 'subsample': 0.9401102150573821, 'colsample_bytree': 0.9014719637619254, 'min_child_samples': 23}. Best is trial 1 with value: 103.63078717089086.


...

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005558 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3725
[LightGBM] [Info] Number of data points in the train set: 77925, number of used features: 24
[LightGBM] [Info] Start training from score 24.539766
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leave

[I 2025-02-14 12:02:18,841] Trial 48 finished with value: 92.36356202052113 and parameters: {'n_estimators': 121, 'max_depth': 9, 'learning_rate': 0.18175431344101442, 'subsample': 0.7986838997710017, 'colsample_bytree': 0.9072699270266997, 'min_child_samples': 21}. Best is trial 33 with value: 84.66438724862945.


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010851 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3725
[LightGBM] [Info] Number of data points in the train set: 77925, number of used features: 24
[LightGBM] [Info] Start training from score 24.539766
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

[I 2025-02-14 12:02:51,747] Trial 49 finished with value: 88.24300600225374 and parameters: {'n_estimators': 842, 'max_depth': 8, 'learning_rate': 0.2910856149639349, 'subsample': 0.8373248535734683, 'colsample_bytree': 0.9360512799582174, 'min_child_samples': 35}. Best is trial 33 with value: 84.66438724862945.


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Best Hyperparameters: {'n_estimators': 922, 'max_depth': 9, 'learning_rate': 0.12014566919484884, 'subsample': 0.9033537815012155, 'colsample_bytree': 0.9130915448535754, 'min_child_samples': 23}
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006398 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3725
[LightGBM] [Info] Number of data points in the train set: 77925, number of used features: 24
[LightGBM] [Info] Start training from score 2

#### XGBoost

In [52]:
# # Fungsi evaluasi MSE
# def mse(y_true, y_pred):
#     return mean_squared_error(y_true, y_pred)

# # Definisikan objective function untuk Optuna
# def objective(trial):
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
#         'max_depth': trial.suggest_int('max_depth', 3, 9),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
#         'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
#         'gamma': trial.suggest_float('gamma', 0.0, 5.0)
#     }
    
#     # Out-of-fold prediction
#     oof_predictions = np.zeros(len(result))
#     kf = KFold(n_splits=10, shuffle=True, random_state=42)
    
#     for fold, (data_idx, val_idx) in enumerate(kf.split(result)):
#         X_data, X_val = result.drop('GT_NO2', axis=1).iloc[data_idx], result.drop('GT_NO2', axis=1).iloc[val_idx]
#         y_data, y_val = result['GT_NO2'].iloc[data_idx], result['GT_NO2'].iloc[val_idx]
        
#         model = xgb.XGBRegressor(**params, objective='reg:squarederror', seed=42)
#         model.fit(X_data, y_data)
        
#         oof_predictions[val_idx] = model.predict(X_val)
    
#     return mse(result['GT_NO2'], oof_predictions)

# # Lakukan optimasi hyperparameter
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=50)

# # Gunakan parameter terbaik untuk model final
# best_params = study.best_params
# print(f"Best Hyperparameters: {best_params}")

# xgb_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror', seed=42)
# y = result['GT_NO2']

# # Out-of-fold predictions dengan parameter terbaik
# oof_predictions = np.zeros(len(result))
# kf = KFold(n_splits=10, shuffle=True, random_state=42)
# fold_scores = []

# for fold, (data_idx, val_idx) in enumerate(kf.split(result)):
#     X_data, X_val = result.drop('GT_NO2', axis=1).iloc[data_idx], result.drop('GT_NO2', axis=1).iloc[val_idx]
#     y_data, y_val = result['GT_NO2'].iloc[data_idx], result['GT_NO2'].iloc[val_idx]
    
#     xgb_model.fit(X_data, y_data)
#     val_pred = xgb_model.predict(X_val)
#     oof_predictions[val_idx] = val_pred
    
#     fold_mse = mse(y_val, val_pred)
#     fold_scores.append(fold_mse)
#     print(f"Fold {fold+1} MSE: {fold_mse:.4f}")

# print(f"\nRata-rata MSE: {np.mean(fold_scores):.4f}")


---
## test data

In [53]:
# Buat klastering KMeans dengan 3 klaster
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
dtest['Kmeans'] = kmeans.fit_predict(dtest[['NO2_strat', 'NO2_total', 'NO2_trop']])

# Normalisasi TropopausePressure
scaler = StandardScaler()
dtest['TropopausePressure'] = scaler.fit_transform(dtest[['TropopausePressure']])

# Polynomial Features untuk 'Precipitation', 'LST', 'AAI'
poly = PolynomialFeatures(degree=2)
poly_features = poly.fit_transform(dtest[['Precipitation', 'LST', 'AAI']])

# Dapatkan nama kolom hasil transformasi polinomial
poly_feature_columns = poly.get_feature_names_out(['Precipitation', 'LST', 'AAI'])

# Membuat DataFrame baru dari fitur polinomial
poly_dtest = pd.DataFrame(poly_features, columns=poly_feature_columns)

# Hapus fitur yang tidak diperlukan
poly_dtest.drop(['1', 'Precipitation', 'LST', 'AAI'], axis=1, inplace=True)

# Gabungkan dengan dtest asli
result_test = pd.concat([dtest, poly_dtest], axis=1)

# Pastikan test.index adalah datetime
result_test["month"] = test.index.month
result_test["year"] = test.index.year
result_test["day"] = test.index.day

# Sinusoidal Encoding Bulan
result_test['month_Sin'] = np.sin(2 * np.pi * result_test['month'] / 12)
result_test['month_Cos'] = np.cos(2 * np.pi * result_test['month'] / 12)

# Hitung elapsed months berdasarkan tahun minimum di result_test
min_year = result_test['year'].min()
result_test['Elapsed_months'] = (result_test['year'] - min_year) * 12 + result_test['month']

# Quarter & Semester
result_test['Quarter'] = ((result_test['month'] - 1) // 3 + 1).astype(int)
result_test['Semester'] = ((result_test['month'] - 1) // 6 + 1).astype(int)

# Identifikasi Tahun-Bulan dalam format YYYYMM
result_test['yearmonth'] = (result_test['year'] * 100 + result_test['month']).astype(int)


In [54]:
result_test.head()

,Precipitation,LST,AAI,CloudFraction,NO2_strat,NO2_total,NO2_trop,TropopausePressure,Kmeans,Precipitation^2,Precipitation LST,Precipitation AAI,LST^2,LST AAI,AAI^2,month,year,day,month_Sin,month_Cos,Elapsed_months,Quarter,Semester,yearmonth
0,3.277529,281.8314,-0.313361,0.771456,0.000024,0.000075,0.000048,-0.862889,0,10.742196,923.710587,-1.027051,79428.938026,-88.315104,0.098195,1,2019,1,0.5,0.866025,1,1,1,201901
1,0.000000,280.7216,-0.229512,0.398208,0.000023,0.000120,0.000073,-0.864959,0,0.000000,0.000000,-0.000000,78804.616707,-64.429036,0.052676,1,2019,1,0.5,0.866025,1,1,1,201901
2,0.000000,282.9800,-0.470822,0.153694,0.000023,0.000171,0.000148,-0.867253,1,0.000000,0.000000,-0.000000,80077.680400,-133.233287,0.221674,1,2019,1,0.5,0.866025,1,1,1,201901
3,1.928031,278.0236,0.132952,0.756917,0.000024,0.000266,0.000201,-0.861829,1,3.717303,536.038078,0.256336,77297.122157,36.963880,0.017676,1,2019,1,0.5,0.866025,1,1,1,201901
4,0.000000,280.0018,-0.198272,0.678858,0.000023,0.000149,0.000124,-0.862601,0,0.000000,0.000000,-0.000000,78401.008003,-55.516618,0.039312,1,2019,1,0.5,0.866025,1,1,1,201901


In [55]:
result_test['target'] = lgbm.predict(result_test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [57]:
submission = pd.read_csv('raw_dataset/SampleSubmission.csv')

submission['GT_NO2'] = result_test['target']
submission

,ID_Zindi,GT_NO2
0,ID_003WOR,25.881186
1,ID_009Y2B,34.035146
2,ID_01C05M,32.008615
3,ID_0216G3,30.897995
4,ID_023C6G,31.903755
...,...,...
6571,ID_ZVEQ52,36.314812
6572,ID_ZWZ1IA,44.422341
6573,ID_ZX7B4A,25.150565
6574,ID_ZYZLPS,36.182595


In [58]:
submission.to_csv('Submission/Submission_LGBM.csv', index=False)
testing = pd.read_csv('Submission/Submission_LGBM.csv')
testing.head()

,ID_Zindi,GT_NO2
0,ID_003WOR,25.881186
1,ID_009Y2B,34.035146
2,ID_01C05M,32.008615
3,ID_0216G3,30.897995
4,ID_023C6G,31.903755
